# Teleportation Algorithm
<p> This module is designed to implement the Quantum Teleportation algorithm, as well as some additional functions that help measure qubits. I use D-Wave Gates Package, to easily implement the low-level gates necessary for teleportation. </p>

In [5]:
"""
Author: Jack Waslen
Date: April 2024
"""

import dwave.gate.operations as gate
from dwave.gate.simulator import simulate
from dwave.gate import Circuit
from dwave.gate.registers.registers import ClassicalRegister, QuantumRegister
from math import pi
import numpy as np

> I start by initializing some helper functions, to help me perform the measurements in the protocol. I can map them to one of three bit combinations. I think I'm going to generalize them later, to work for larger numbers of qubits, but for now these implementations suit my needs.

In [6]:
three_bits = ["000","001","010","011","100","101","110","111"]
two_bits = ["00","01","10","11"]
one_bits = ["0","1"]


def measureFirstQubit(circuit_state):
    qubit_factors = circuit_state
    factors = absolute_square(qubit_factors)
    outcomes = {"0":0,"1":0}
    
    i = 0
    for value in factors.values():
        if i < 4:
            # I round the probabilities off due to binary limitations with float numbers.
            outcomes["0"] += round(value,3)
        else:
             # I round the probabilities off due to binary limitations with float numbers.
            outcomes["1"] += round(value,3)
        i += 1
    return normalization(outcomes)


def normalization(probabilities):
    N = 0
    
    # Adjusting function based on if were calculating one bit, two bits or three bits.
    possible_values = three_bits
    if len(probabilities) == 4:
        possible_values = two_bits
    if len(probabilities) == 2:
        possible_values = one_bits
    
    # Adding up the prexisting probablities into their normalization factor.
    for combo in possible_values:
        N += probabilities[combo]
    
    # If the probabilities are not normalized, normalize them.
    if N != 1:
        for combo in possible_values:
            probabilities[combo] = probabilities[combo] / N
            
    return probabilities
                
def absolute_square(factors):
    absolute_squares = {}
    
    # Adjusting function based on if were calculating one bit, two bits or three bits.
    bit_combinations = three_bits
    
    if len(factors) == 2:
        bit_combinations = one_bits
    if len(factors) == 4:
        bit_combinations = two_bits
    
    # Loop through mapping probabilities to bit values
    for index in range(len(factors)):
        probability_amplitude = abs(factors[index].real) ** 2 + abs(factors[index].imag) ** 2
        absolute_squares[bit_combinations[index]] = round(probability_amplitude,3)

    return absolute_squares


> In order for the teleportation protocol to work, you simply pass in the probability of 1, and the protocol performs the required rotation on the input qubit.

In [7]:
def teleportation(probability):
    # Circuit starts in state |000> with 3 qubits
    circuit = Circuit(3, 3)
    
    with circuit.context as (qubits, cbits):
        # qubit[0] = Alice's First Qubit, qubit[1] = Alice's Second Qubit, qubit[2] = Bob's Qubit

        # This section sets the value of Ψ (currently qubit[0]).
        angle = 2 * np.arcsin(np.sqrt(probability))
        gate.Rotation(parameters=[0,angle,0], qubits=qubits[0])

        # Simulate runs the circuit, ensures all gates are applied
        simulate(circuit)
        
        print("The value of Ψ is: "+str(measureFirstQubit(circuit.state)))

        # Teleportation Protocol
        gate.Hadamard(qubits[1])
        gate.CX(control=qubits[1],target=qubits[2])
        gate.CX(control=qubits[0],target=qubits[1])
        gate.Hadamard(qubits[0])

        # Simulate runs the circuit, ensures all gates are applied
        simulate(circuit)
        
        print("\n")
    
        print("The Factors for all Eight Combinations: (000, 001, 010 ... 111)")
        print("\n")

        # Now we measure Alice's Qubits.
        for combo in two_bits:
            print("The Payment Machine measures: "+combo)
            measured = combo
            # Measure Bob's Qubit accordingly
            measureBob(measured, circuit, qubits)
teleportation(0.1)

The value of Ψ is: {'0': 0.9, '1': 0.1}


The Factors for all Eight Combinations: (000, 001, 010 ... 111)


The Payment Machine measures: 00
Bank server measures: 
{'0': 0.9, '1': 0.1}
We have teleported qubit Ψ to the Bank Server!


The Payment Machine measures: 01
Bank server applies X and measures: 
{'0': 0.9, '1': 0.1}
We have teleported qubit Ψ to the Bank Server!


The Payment Machine measures: 10
Bank server applies Z and measures: 
{'0': 0.9, '1': 0.1}
We have teleported qubit Ψ to the Bank Server!


The Payment Machine measures: 11
Bank server applies X and Z and measures: 
{'0': 0.9, '1': 0.1}
We have teleported qubit Ψ to the Bank Server!




> This function deals with the gates applied to Bob's Qubit, then measures and records the result before resetting the qubit.

In [2]:
def measureBob(measured, circuit, qubits):
    if measured[0:2] == "00":  
        simulate(circuit)
        alpha_and_beta = normalization(absolute_square(circuit.state[0:2]))
        print("Bank server measures: ")
   
        print(alpha_and_beta)
        print("We have teleported qubit Ψ to the Bank Server!")
        print("\n")

        
    if measured[0:2] == "01":
        gate.X(qubits[2])
        simulate(circuit)
        
        alpha_and_beta = normalization(absolute_square(circuit.state[2:4]))
        print("Bank server applies X and measures: ")
        print(alpha_and_beta)
        print("We have teleported qubit Ψ to the Bank Server!")
        print("\n")

        
        # Reset Circuit for future runs
        gate.X(qubits[2])
    
    if measured[0:2] == "10":
        gate.Z(qubits[2])
        simulate(circuit)
        
        alpha_and_beta = normalization(absolute_square(circuit.state[4:6]))
        print("Bank server applies Z and measures: ")
        print(alpha_and_beta)
        print("We have teleported qubit Ψ to the Bank Server!")
        print("\n")
 
        
        # Reset Circuit for future runs
        gate.Z(qubits[2])
        
    if measured[0:2] == "11":
        gate.X(qubits[2])
        gate.Z(qubits[2])
        simulate(circuit)
    
        alpha_and_beta = normalization(absolute_square(circuit.state[6:8]))
        print("Bank server applies X and Z and measures: ")
        print(alpha_and_beta)
        print("We have teleported qubit Ψ to the Bank Server!")
        print("\n")

        
        # Reset Circuit for future runs
        gate.Z(qubits[2])
        gate.X(qubits[2])
      
